In [ ]:
This pipeline could help crop images to circle size, and analyze the result.

In [1]:
import boto3
import io
from PIL import Image,ImageDraw
import s3fs
fs = s3fs.S3FileSystem()
s3 = boto3.client('s3', region_name='us-east-2')
import tensorflow as tf
import pandas as pd
from tensorflow.contrib.predictor import from_saved_model
import s3fs
import pandas as pd
import numpy as np

In [2]:
def crop_save(s3, old_path:tuple, new_path:tuple):
    '''
    s3: reference of s3 client
    old_path = (old_bucket,old_key)
    new_path = (new_bucket,new_key)
    kcrop = {'l':rate0,'t':rate1,'r':rate2,'b':rate3}
    '''
    down_stream = s3.get_object(Bucket=old_path[0],Key=old_path[1])['Body']
    im = Image.open(down_stream)
    npImage=np.array(im)
    h,w=im.size
    radius=0.37
    x0 = 0.5*h-radius*w
    y0 = (0.5-radius)*w
    x1 = 0.5*h + radius*w
    y1 = (0.5+radius)*w
    alpha = Image.new('L', im.size,0)
    draw = ImageDraw.Draw(alpha)
    draw.pieslice([x0,y0,x1,y1],0,360,fill=255)
    npAlpha=np.array(alpha)
    npImage=np.dstack((npImage,npAlpha))
    im1 = Image.fromarray(npImage)
    up_stream = io.BytesIO()
    im1.save(up_stream, format='png')
    up_stream.seek(0)
    s3.upload_fileobj(up_stream, new_path[0], new_path[1])
    down_stream.close()
    up_stream.close()
    return 

In [3]:
for old_path in fs.ls('Object Name')[1:]:
    pos = old_path.find('/')
    old_path = (old_path[:pos],old_path[pos+1:])
    k = old_path[1]
    p = k.rfind('/')
    new_path = ('S3 Bucket','Object Name'+'cropped_'+k[p+1:])
    crop_save(s3, old_path, new_path)

In [4]:
df = fs.ls('Bucket Name')[1:]

In [ ]:
df

In [6]:
model=from_saved_model("model")

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from model/variables/variables


In [ ]:
my_list = [] #create my_list to append result
for file in fs.ls('Bucket Name')[1:]:
    fp = fs.open('s3://' + file) #open file
    data = {'image_bytes': [fp.read()]} #read image
    results = model(data) #call the model
    final_result = results['classes'][0][0] #Slicing the first result
    if final_result.decode("utf-8").find('Crystal') != -1: #transfer the format of result
        my_list.append(str(1))
    else:
        my_list.append(str(0))

In [ ]:
my_list

In [19]:
result_list = pd.DataFrame(my_list, columns = ['classify']) #transfer the list to dataframe

In [ ]:
df = pd.DataFrame(df, columns = ['id']) #list the file
df_final = pd.concat([df, result_list], axis = 1) #concat two dataframes
df_final #final result

In [21]:
df_final.to_excel("after_circle_crop.xlsx")

In [22]:
df_final['classify'].value_counts()

1    71
0    35
Name: classify, dtype: int64